# Lets Begin

## Decrypt the current data

In [ ]:
from cryptography.fernet import Fernet
import os

folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

encrypt_dir = "current_data"
decrypt_dir = "current_data"

# Ensure the encrypt directory exists
os.makedirs(decrypt_dir, exist_ok=True)

# Encrypt each file in the current_data directory
for filename in os.listdir(encrypt_dir):
    file_path = os.path.join(encrypt_dir, filename)
    
    # Read the file data
    with open(file_path, "rb") as file:
        file_data = file.read()

    # Decrypt data
    encrypted_data = f.decrypt(file_data)

    # Write the encrypted file to the encrypt directory
    encrypted_file_path = os.path.join(decrypt_dir, filename)
    with open(encrypted_file_path, "wb") as file:
        file.write(encrypted_data)

print("Decryptiong complete")

## Ingest new IKBR CSV report
Make sure to add the report into IB_export folder and name it with ib_DDMMYYYY.csv. Replace the value of `ib_extract_file_name` with the name of the file you just added. 

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format ='retina'

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

ib_extract_file_name = "2025-1Jan-6Jul.csv"
source_folder = "IB_extract"
target_folder = "current_data"

"""
Function to Extract respective sections from IB extract csv file
"""
def extract(section_name, section_description):
    with open(f'{source_folder}/{ib_extract_file_name}', 'r') as source_file:
        lines = source_file.readlines()

    start_index = None
    end_index = None
    bos = f'"BOS","{section_name}","{section_description}"'
    eos = f'"EOS","{section_name}"'

    for i, line in enumerate(lines):
        if bos in line:
            start_index = i + 1
        if eos in line and start_index is not None:
            end_index = i - 1
            break

    if start_index is not None and end_index is not None:
        with open(f'{source_folder}/temp.csv', 'w') as positions_file:
            for line in lines[start_index:end_index + 1]:
                positions_file.write(line)

### Extract Cash Report (CRTT)

In [ ]:
section_name = "CRTT"
extract(section_name, "Cash Report; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "LevelOfDetail",
        "FromDate", 
        "ToDate", 
        "Commissions",
        "Deposits",
        "Withdrawals",
        "BrokerInterest",
        "NetTradesSales",
        "NetTradesPurchases",
        "SalesTax",
        "EndingCash",
        "EndingSettledCash",
        "NetCashBalanceSLB",
        "NetSettledCashBalanceSLB",
    ]

data = pd.read_csv(f'{source_folder}/temp.csv')
cash_report = data[columns].copy()
cash_report.to_csv(f'{target_folder}/{section_name}.csv', index=False)

### Extract Cash Transactions (CTRN)

In [ ]:
section_name = "CTRN"
extract(section_name, "Cash Transactions")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "FXRateToBase",
        "Date/Time", 
        "SettleDate", 
        "Amount",
        "Type",
        "TransactionID",
        "ReportDate",
    ]

data = pd.read_csv(f'{source_folder}/temp.csv')
source_ctrn = data[columns].copy()

if os.path.exists(f'{target_folder}/{section_name}.csv'):
    data = pd.read_csv(f'{target_folder}/{section_name}.csv')
    current_ctrn = data[columns].copy()

    # Find TransactionIDs that are in source_ctrn but not in current_ctrn
    new_trades = source_ctrn[~source_ctrn['TransactionID'].isin(current_ctrn['TransactionID'])]

    # Append new transactions to current_ctrn
    updated_trades = pd.concat([current_ctrn, new_trades])

    # Write the updated dataframe to final.csv
    updated_trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)
else:
    source_ctrn.to_csv(f'{target_folder}/{section_name}.csv', index=False)


### Positions (POST)

In [ ]:
section_name = "POST"
extract(section_name, "Position; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "AssetClass",
        "Symbol", 
        "Description", 
        "Conid",
        "SecurityID",
        "ListingExchange",
        "IssuerCountryCode",
        "ReportDate",
        "Quantity",
        "CostBasisPrice",
        "CostBasisMoney",
        "PercentOfNAV",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
positions = data[columns].copy()
positions.to_csv(f'{target_folder}/{section_name}.csv', index=False)

### Trades (TRNT)

In [ ]:
section_name = "TRNT"
extract(section_name, "Trades; trade date basis")

columns=[
        "ClientAccountID",
        "CurrencyPrimary",
        "AssetClass",
        "Symbol", 
        "Description", 
        "Conid",
        "SecurityID",
        "ListingExchange",
        "IssuerCountryCode",
        "TradeID",
        "TradeDate",
        "SettleDateTarget",
        "TransactionType",
        "Exchange",
        "Quantity",
        "TradePrice",
        "TradeMoney",
        "Taxes",
        "IBCommission",
        "NetCash",
        "CostBasis",
        "FifoPnlRealized",
        "Buy/Sell",
        "LevelOfDetail",
        "OrderTime",
    ]
data = pd.read_csv(f'{source_folder}/temp.csv')
trades = data[columns].copy()


if os.path.exists(f'{target_folder}/{section_name}.csv'):
    data = pd.read_csv(f'{target_folder}/{section_name}.csv')
    current_trades = data[columns].copy()

    key_cols = ['TradeDate', 'Quantity', 'TradePrice', 'OrderTime', 'Buy/Sell', 'LevelOfDetail']
    new_trades = trades[~trades[key_cols].apply(tuple, axis=1).isin(current_trades[key_cols].apply(tuple, axis=1))]
    updated_trades = pd.concat([current_trades, new_trades])
    updated_trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)
else:
    trades.to_csv(f'{target_folder}/{section_name}.csv', index=False)

#### Encrypt data before code commit

In [ ]:
from cryptography.fernet import Fernet
import os

folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

# Define the directories
current_data = "current_data"
IB_extract = "IB_extract"

def encrypt_file(directory, file_name):
    file_path = os.path.join(directory, file_name)
    with open(file_path, "rb") as file:
        file_data = file.read()

    encrypted_data = f.encrypt(file_data)

    with open(file_path, "wb") as file:
        file.write(encrypted_data)

# Encrypt each file in the current_data directory
for filename in os.listdir(current_data):
    encrypt_file(current_data, filename)

# Encrypt IB_extract report
encrypt_file(IB_extract, ib_extract_file_name)

# Encrypt temp file
encrypt_file(IB_extract, "temp.csv")

print("Encryption completed")



#### ⚠️ Decrypt
Run this to selectively decrypt files. Ensure to encrypt them back before commit...

In [ ]:
folio_key = os.getenv("FOLIO_KEY")

f = Fernet(folio_key)

encrypt_dir = "temp"
decrypt_dir = "temp"

# Ensure the encrypt directory exists
os.makedirs(decrypt_dir, exist_ok=True)

# Encrypt each file in the current_data directory
for filename in os.listdir(encrypt_dir):
    file_path = os.path.join(encrypt_dir, filename)
    
    # Read the file data
    with open(file_path, "rb") as file:
        file_data = file.read()

    # Decrypt data
    encrypted_data = f.decrypt(file_data)

    # Write the encrypted file to the encrypt directory
    encrypted_file_path = os.path.join(decrypt_dir, filename)
    with open(encrypted_file_path, "wb") as file:
        file.write(encrypted_data)

print("Decryptiong complete")

## 🛑 Generate Lock File
YOU SHOULD NOT NEED THIS ANYMORE

In [ ]:
from cryptography.fernet import Fernet

# Create new key - YOU SHOULD NOT NEED THIS ANYMORE
# def write_key():
#     key = Fernet.generate_key()
#     with open("folio.key", "wb") as key_file:
#         key_file.write(key)

# write_key()